In [1]:
import tensorflow as tf
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.preprocessing import LabelBinarizer
import keras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import random
from keras.applications import EfficientNetB0

# Load and Process Data

Data Source: https://www.kaggle.com/datasets/gpiosenka/100-bird-species

Initial data had 525 species of birds with 100 to 200 images per species in the training set. To make the training times more reasonable, we reduced the data down to 50 randomly selected species and only the first 100 training images for each species.

In [2]:
# Set file path for loading data
train_dir = "C:/Users/bcbot/Deep-Learning/birds/train"
val_dir = "C:/Users/bcbot/Deep-Learning/birds/valid"

In [3]:
# Instantiate the containers for holding image and label data
train_data = []
val_data = []

# Load the first 100 files for each bird for training set
for i in os.listdir(train_dir):
    sub_directory = os.path.join(train_dir, i)
    for j in os.listdir(sub_directory):
        img = cv2.imread(os.path.join(sub_directory, j))
        train_data.append([img, i])

# Load first 50 files for each bird in validation set
for i in os.listdir(val_dir):
    sub_directory = os.path.join(val_dir, i)
    for j in os.listdir(sub_directory):
        img = cv2.imread(os.path.join(sub_directory, j))
        val_data.append([img, i])

print(len(train_data))
print(len(val_data))

5000
250


In [4]:
# Validating that all images are the same shape

count_t = 0
count_v = 0

for t in train_data:
    if t[0].shape != (224, 224, 3):
        count_t += 1
    
for v in val_data:
    if v[0].shape != (224, 224, 3):
        count_v += 1

print(f'There are {count_t} images of different shape in the train data.\nThere are {count_v} images of different shape in the validation data.')
    

There are 0 images of different shape in the train data.
There are 0 images of different shape in the validation data.


In [5]:
# Shuffle data

np.random.seed(7284)
np.random.shuffle(train_data)
np.random.shuffle(val_data)

In [6]:
# Preprocess training data and validation data
lb = LabelBinarizer()

X_train = []
y_train = []
for x, y in train_data:
    X_train.append(x)
    y_train.append(y)

X_train = np.array(X_train)
y_train = np.array(y_train)

y_train_vect = lb.fit_transform(y_train)

X_val = []
y_val = []
for x, y in val_data:
    X_val.append(x)
    y_val.append(y)

X_val = np.array(X_val)
y_val = np.array(y_val)

y_val_vect = lb.fit_transform(y_val)

# Evaluate on Baseline Model

The dataset authros noted that they had achieved good success using EfficientNetB0, so we decided to use that as our baseline model.

In [7]:
# Set early stopping parameters
callback = EarlyStopping(monitor= 'val_accuracy', patience= 5, start_from_epoch= 5)

In [8]:
# Load Model 
enet = EfficientNetB0(include_top= False, input_shape=(224, 224, 3))

layers = enet.layers

for layer in layers:
    layer.trainable = False

# Add a flatten and softmax layer
model= Sequential([
    enet,
    Flatten(),
    Dense(50, activation='softmax')
])

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
history = model.fit(x = X_train, y= y_train_vect, batch_size=80, validation_data= (X_val, y_val_vect), verbose = 1, epochs = 30, callbacks = [callback])

Epoch 1/30


63/63 [==============================] - 41s 591ms/step - loss: 1.9436 - accuracy: 0.7448 - val_loss: 0.2981 - val_accuracy: 0.9360
Epoch 2/30
63/63 [==============================] - 35s 551ms/step - loss: 0.2495 - accuracy: 0.9592 - val_loss: 0.4441 - val_accuracy: 0.9280
Epoch 3/30
63/63 [==============================] - 36s 573ms/step - loss: 0.1228 - accuracy: 0.9782 - val_loss: 0.5983 - val_accuracy: 0.9240
Epoch 4/30
63/63 [==============================] - 35s 558ms/step - loss: 0.0687 - accuracy: 0.9878 - val_loss: 0.4944 - val_accuracy: 0.9480
Epoch 5/30
63/63 [==============================] - 35s 554ms/step - loss: 0.0984 - accuracy: 0.9846 - val_loss: 0.3774 - val_accuracy: 0.9360
Epoch 6/30
63/63 [==============================] - 36s 567ms/step - loss: 0.0558 - accuracy: 0.9886 - val_loss: 0.5302 - val_accuracy: 0.9400
Epoch 7/30
63/63 [==============================] - 37s 588ms/step - loss: 0.0770 - accuracy: 0.9874 - val_loss: 0.4072 - val_accuracy: 0.95

# Check for Problem Difficulty

To ensure the smaller dataset didn't make the problem too easy, we ran it through a very simple model to see what a baseline starting point for our built-from-scratch model would be

In [10]:
# Model with 1 CNN layer, 1 pooling layer, and a softmax layer
model = Sequential([
    Conv2D(32, 3, input_shape = (224, 224, 3), activation= 'relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(50, activation='softmax')
])

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
history2 = model.fit(x = X_train/255, y= y_train_vect, batch_size=80, validation_data= (X_val/255, y_val_vect), verbose = 1, epochs = 30, callbacks = [callback])

Epoch 1/30
63/63 [==============================] - 17s 259ms/step - loss: 8.5220 - accuracy: 0.1320 - val_loss: 2.5930 - val_accuracy: 0.3840
Epoch 2/30
63/63 [==============================] - 16s 260ms/step - loss: 1.5623 - accuracy: 0.6462 - val_loss: 1.6337 - val_accuracy: 0.5480
Epoch 3/30
63/63 [==============================] - 16s 261ms/step - loss: 0.3205 - accuracy: 0.9398 - val_loss: 1.5852 - val_accuracy: 0.5600
Epoch 4/30
63/63 [==============================] - 16s 256ms/step - loss: 0.0561 - accuracy: 0.9956 - val_loss: 1.4997 - val_accuracy: 0.6120
Epoch 5/30
63/63 [==============================] - 16s 260ms/step - loss: 0.0182 - accuracy: 0.9992 - val_loss: 1.4290 - val_accuracy: 0.6160
Epoch 6/30
63/63 [==============================] - 16s 255ms/step - loss: 0.0057 - accuracy: 1.0000 - val_loss: 1.4246 - val_accuracy: 0.6360
Epoch 7/30
63/63 [==============================] - 17s 263ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 1.4353 - val_accuracy: 0.6400